## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

REDIS_KEY = os.environ["REDIS_KEY"]
REDIS_HOST = os.environ["REDIS_HOST"]

redisConnString = 'rediss://default:{}@{}:6380/0'.format(REDIS_KEY,REDIS_HOST)
sessionId = 'test-session-id'

## Initiate Redis

In [2]:
from langchain.memory import RedisChatMessageHistory

history = RedisChatMessageHistory(
    url=redisConnString,
    session_id=sessionId,
    ttl=300 # 5 minutes
)

history.clear()
history.add_ai_message("Hello, I am an AI assistant.")
history.add_user_message("Hello, I'm Kazuya.")
history.add_ai_message("Hi Kazuya, how can I help you today?")

In [3]:
# Test memory storage, try retrieving the history from Redis.
retrievedHistory = RedisChatMessageHistory(
    url=redisConnString,
    session_id=sessionId,
    ttl=300 # 5 minutes
)

retrievedHistory.messages

[AIMessage(content='Hello, I am an AI assistant.', additional_kwargs={}, example=False),
 HumanMessage(content="Hello, I'm Kazuya.", additional_kwargs={}, example=False),
 AIMessage(content='Hi Kazuya, how can I help you today?', additional_kwargs={}, example=False)]

## LangChain Redis Memory

In [4]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    max_tokens=50,
    temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    max_tokens=100,
    temperature=0.3,
    verbose=True
)

conversation = ConversationChain(
    llm=chat,
    verbose=True)

> **WARNING: ConversationSummaryBufferMemory does not work well with persistent memory storage at this time.**
> Convert this block to code when [this issue](https://github.com/langchain-ai/langchain/issues/3072) is fixed

```python
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    chat_memory=retrievedHistory
    return_messages=False
)

conversation.memory = memory

conversation.predict(input="Can you tell me about our health benefits?")
conversation.predict(input="Can I claim for a new pair of glasses?")
conversation.predict(input="How about skin moisturizers?")
```

In [5]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k = 2,
    chat_memory=retrievedHistory
)

conversation.memory = memory
conversation.predict(input="Can you tell me about our health benefits?")
conversation.predict(input="Can I claim for a new pair of glasses?")
conversation.predict(input="How about skin moisturizers?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI: Hello, I am an AI assistant.
Human: Hello, I'm Kazuya.
AI: Hi Kazuya, how can I help you today?
Human: Can you tell me about our health benefits?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I'm Kazuya.
AI: Hi Kazuya, how can I help you today?
Human: Can you tell me about our health benefits?
AI: Sure, Kazuya. Our health benefits include medical, dental, a

"I'm sorry, Kazuya, but I do not have information on whether skin moisturizers are covered under our health benefits. Would you like me to look up more information on this topic?"

## Check Redis

In [6]:
# Test memory storage, try retrieving the history from Redis.
latestHistory = RedisChatMessageHistory(
    url=redisConnString,
    session_id=sessionId,
    ttl=300 # 5 minutes
)

latestHistory.messages

[AIMessage(content='Hello, I am an AI assistant.', additional_kwargs={}, example=False),
 HumanMessage(content="Hello, I'm Kazuya.", additional_kwargs={}, example=False),
 AIMessage(content='Hi Kazuya, how can I help you today?', additional_kwargs={}, example=False),
 HumanMessage(content='Can you tell me about our health benefits?', additional_kwargs={}, example=False),
 AIMessage(content='Sure, Kazuya. Our health benefits include medical, dental, and vision coverage. We also offer a wellness program that includes gym memberships, health coaching, and mental health resources. Our medical coverage includes preventative care, prescription drug coverage, and specialist visits. Our dental coverage includes cleanings, fillings, and major procedures like root canals and crowns. Our vision coverage includes eye exams, glasses, and contacts. Is there anything specific you would like me to look up or clarify?', additional_kwargs={}, example=False),
 HumanMessage(content='Can I claim for a new 